In [2]:
import pandas as pd
import matplotlib.pyplot as mltp
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import sklearn as sk
import seaborn as sea
import re 
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()

/Users/niclascramer/opt/miniconda3/envs/NLP/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
import numpy as np
import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import pickle 

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

/Users/niclascramer/opt/miniconda3/envs/NLP/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


### Usage

1. Have a list of lemitised words of the text 
2. CSV of the Words with there respected wahrscheinlichkeit

In [19]:
def create_df_overview(data_words,num_topics,df_topic):
    topic_of_text=[]
    for text in tqdm(data_words):
        liste_text_top=[]
        for t in range(num_topics):
            liste_top=[]
            for element in df_topic[f'{t}_word'].tolist():
                liste_top.append(len([i for i, e in enumerate(text) if e == element]))
    
            Result = []
            for i1, i2 in zip(liste_top, df_topic[f'{t}_per'].tolist()):
                Result.append(i1*i2)
            liste_text_top.append(sum(Result))    
        if sum(liste_text_top)!=0:
            topic_of_text.append([text,max(range(len(liste_text_top)), key=liste_text_top.__getitem__),max(liste_text_top)/sum(liste_text_top),liste_text_top])
        else:
            topic_of_text.append([text,'None','None',liste_text_top])
    return pd.DataFrame(data=topic_of_text,columns=['text','topic_num','percent','perc_list'])

In [20]:
df_text_spacy=pd.read_csv('data/train_500.csv')
df_text_spacy.head()

,Unnamed: 0,id,date,text,lang,language_2,word_tokenize,sent_tokenize,count_word,count_sent,word_tokenize_num_of_stopwords,word_tokenize_without_stopwords,word_tokenize_without_stopwords_port,msg_lemmatized,cleanLinks,word_per_sent_mean,gender,age,topic,sign
0,445164,2970791,"25,May,2004",This blog is being posted due to the fact that...,en,en,"['This', 'blog', 'is', 'being', 'posted', 'due...",['This blog is being posted due to the fact th...,83,4,34,"['This', 'blog', 'posted', 'due', 'fact', 'lit...","['thi', 'blog', 'post', 'due', 'fact', 'littl'...","['thi', 'blog', 'post', 'due', 'fact', 'littl'...",This blog is being posted due to the fact that...,20.75,male,14,Student,Sagittarius
1,293091,3931851,"02,agosto,2004",So I have a big fucking interview tomorrow for...,en,en,"['So', 'I', 'have', 'a', 'big', 'fucking', 'in...",['So I have a big fucking interview tomorrow f...,88,5,31,"['So', 'I', 'big', 'fucking', 'interview', 'to...","['so', 'i', 'big', 'fuck', 'interview', 'tomor...","['so', 'i', 'big', 'fuck', 'interview', 'tomor...",So I have a big fucking interview tomorrow for...,17.60,male,15,Student,Pisces
2,594697,3481650,"07,July,2004",I was reminded just now of the time Ashley and...,en,en,"['I', 'was', 'reminded', 'just', 'now', 'of', ...",['I was reminded just now of the time Ashley a...,574,32,234,"['I', 'reminded', 'time', 'Ashley', 'I', 'drov...","['i', 'remind', 'time', 'ashley', 'i', 'drove'...","['i', 'remind', 'time', 'ashley', 'i', 'drove'...",I was reminded just now of the time Ashley and...,17.94,female,17,Student,Gemini
3,3764,4187211,"20,August,2004",I was checking up on my cousin Dylan and Fanni...,en,en,"['I', 'was', 'checking', 'up', 'on', 'my', 'co...","['""I was checking up on my cousin Dylan and Fa...",84,3,32,"['I', 'checking', 'cousin', 'Dylan', 'Fannie',...","['i', 'check', 'cousin', 'dylan', 'fanni', '""s...","['i', 'check', 'cousin', 'dylan', 'fanni', '""s...",I was checking up on my cousin Dylan and Fanni...,28.00,female,23,Student,Taurus
4,162953,3686696,"24,June,2004",for the NME interview click urlLink part 1 and...,en,en,"['for', 'the', 'NME', 'interview', 'click', 'u...",['for the NME interview click urlLink part 1 a...,12,1,3,"['NME', 'interview', 'click', 'urlLink', 'part...","['nme', 'interview', 'click', 'urllink', 'part...","['nme', 'interview', 'click', 'urllink', 'part...",for the NME interview click urlLink part 1 and...,12.00,female,23,Student,Aquarius


In [21]:
#data_words=
df_topic=pd.read_csv('LDA/lda_gesamt.csv')
df_topic=df_topic.loc[:, ~df_topic.columns.str.contains('^Unnamed')]
num_topics=int(len(df_topic.columns)/2)


In [22]:
df_topic.head()

,0_word,0_per,1_word,1_per,2_word,2_per,3_word,3_per,4_word,4_per,...,33_word,33_per,34_word,34_per,35_word,35_per,36_word,36_per,37_word,37_per
0,go,0.176,world,0.123,time,0.023,weird,0.194,free,0.104,...,learn,0.099,nbsp,0.494,note,0.122,add,0.224,link,0.120
1,then,0.069,believe,0.103,good,0.021,brother,0.137,study,0.059,...,however,0.096,comment,0.073,view,0.112,create,0.110,hair,0.077
2,there,0.040,student,0.050,day,0.021,st,0.080,sign,0.058,...,question,0.087,box,0.042,step,0.092,character,0.097,church,0.068
3,guy,0.035,lie,0.044,now,0.020,stare,0.072,level,0.047,...,apparently,0.056,smart,0.032,build,0.065,window,0.064,ill,0.063
4,home,0.034,chance,0.043,really,0.019,universe,0.050,floor,0.042,...,result,0.040,accept,0.030,interested,0.060,difference,0.054,click,0.040


In [23]:
stopwords = stopwords.words('english')

AttributeError: 'list' object has no attribute 'words'

In [24]:
text_liste=df_text_spacy.text.tolist()

In [11]:
def lemmatization_test(texts,  allowed_posttags=['NOUN','ADJ','VERB','ADV']):
    nlp=spacy.load('en_core_web_md',disable=['parser','ner'])
    texts_out=[]
    for text in tqdm(texts):
        doc= nlp(text)
        new_text=[]
        for token in doc:
            if token.pos_ in allowed_posttags:
                new_text.append(token.lemma_)
        final=' '.join(new_text)
        texts_out.append(final)
    return texts_out

In [12]:
lemma_text=lemmatization_test(text_liste)

  0%|          | 0/19500 [00:00<?, ?it/s]

In [13]:
def gen_words(texts):
    final=[]
    for text in tqdm(texts):
        new= gensim.utils.simple_preprocess(text,deacc=True)
        final.append(new)
    return final

In [14]:
data_words=gen_words(lemma_text)

  0%|          | 0/19500 [00:00<?, ?it/s]

In [25]:
create_df_overview(data_words,num_topics,df_topic)

  0%|          | 0/19500 [00:00<?, ?it/s]

,text,topic_num,percent,perc_list
0,"[blog, post, fact, little, development, happen...",7,0.441402,"[0.0, 0.0, 0.16700000000000004, 0.0, 0.021, 0...."
1,"[so, big, fucking, interview, tomorrow, new, s...",20,0.312445,"[0.176, 0.0, 0.083, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
2,"[remind, just, now, time, drive, window, down,...",27,0.203974,"[1.863999999999999, 0.05, 0.5950000000000003, ..."
3,"[check, cousin, wedding, site, just, get, marr...",20,0.278658,"[0.0, 0.0, 0.08300000000000002, 0.08, 0.0, 0.0..."
4,"[interview, click, urllink, part, urllink, part]",31,0.946866,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...
19495,"[urllink, talk, evolutionary, flop, breakfast,...",31,0.240484,"[0.142, 0.24799999999999997, 0.302000000000000..."
19496,"[government, work, pretty, good, website, thin...",31,0.688119,"[0.015, 0.05, 0.045, 0.0, 0.059, 0.0, 0.0, 0.0..."
19497,"[totally, pump, read, book, call, excited, boo...",31,0.436421,"[0.020999999999999998, 0.0, 0.1009999999999999..."
19498,[married],36,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
